In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.metrics import roc_auc_score

## 计算每个模型的auc

In [ ]:
data_y = pd.read_csv('/data/topic1/ensemble_test_y.csv')

In [ ]:
# lgb
data_lgb = pd.read_csv('/data/topic1/ensemble_result_lgb.csv')
data_lgb.rename(columns={'IMEI':'用户标识', 'SCORE':'lgb'}, inplace=True)
ypred = data_lgb['lgb']
print '######## lgb #########'
print metrics.classification_report(y_test, ypred>0.5)
print roc_auc_score(list(data_y['是否去过迪士尼']), ypred)
print '######################'

In [ ]:
# gbdt_lgb
data_gbdt_lgb = pd.read_csv('/data/topic1/ensemble_result_gbdt_lgb.csv')
data_gbdt_lgb.rename(columns={'IMEI':'用户标识', 'SCORE':'gbdt_lgb'}, inplace=True)
ypred = data_gbdt_lgb['gbdt_lgb']
print '######## gbdt_lgb #########'
print metrics.classification_report(y_test, ypred>0.5)
print roc_auc_score(list(data_y['是否去过迪士尼']), ypred)
print '######################'

In [ ]:
# gbdt_xgb
data_gbdt_xgb = pd.read_csv('/data/topic1/ensemble_result_gbdt_xgb.csv')
data_gbdt_xgb.rename(columns={'IMEI':'用户标识', 'SCORE':'gbdt_xgb'}, inplace=True)
ypred = data_gbdt_xgb['gbdt_xgb']
print '######## gbdt_xgb #########'
print metrics.classification_report(y_test, ypred>0.5)
print roc_auc_score(list(data_y['是否去过迪士尼']), ypred)
print '######################'

In [ ]:
# gbdt_lr
data_gbdt_lr = pd.read_csv('/data/topic1/ensemble_result_gbdt_lr.csv')
data_gbdt_lr.rename(columns={'IMEI':'用户标识', 'SCORE':'gbdt_lr'}, inplace=True)
ypred = data_gbdt_lr['gbdt_lr']
print '######## gbdt_lr #########'
print metrics.classification_report(y_test, ypred>0.5)
print roc_auc_score(list(data_y['是否去过迪士尼']), ypred)
print '######################'

## 训练ensemble的分类器

In [ ]:
# 训练
import lightgbm as lgb
from sklearn.model_selection import train_test_split
data = pd.merge(data_lgb, data_gbdt_lgb, on='用户标识')
data = pd.merge(data, data_gbdt_xgb, on='用户标识')
# data = pd.merge(data, data_gbdt_lr, on='用户标识')
X_train, X_test, y_train, y_test = train_test_split(data.drop(['用户标识'], axis=1), data_y, test_size=0.3)

In [ ]:
# lgb做分类器
train_data = lgb.Dataset(X_train.values, label=y_train['是否去过迪士尼'])
test_data = lgb.Dataset(X_test.values, label=y_test['是否去过迪士尼'])
param = {'num_leaves':15, 'num_trees':100, 'objective':' lambdarank', 'metric':'auc'}
# bst=lgb.cv(param, train_data, num_boost_round=100, nfold=3, early_stopping_rounds=30)
# bst = lgb.train(param, train_data, num_boost_round=100)
bst = lgb.train(param, train_data, valid_sets=test_data, num_boost_round=22)
ypred = bst.predict(X_test.values)
print roc_auc_score(y_test['是否去过迪士尼'], ypred)

In [ ]:
# lr做分类器
from sklearn import linear_model
grd_lm = linear_model.LogisticRegressionCV(Cs=10, penalty='l2', tol=1e-6, n_jobs=-1, cv=3)
grd_lm.fit(X_train.values, y_train['是否去过迪士尼'])
ypred = grd_lm.predict_proba(X_test.values)[:, 1]
print roc_auc_score(y_test['是否去过迪士尼'], ypred>0.5)

In [ ]:
# 保存分类器

In [ ]:
# 所有数据,lgb做分类器
train_data = lgb.Dataset(data.drop(['用户标识'], axis=1).values, label=data_y['是否去过迪士尼'])
param = {'num_leaves':15, 'num_trees':100, 'objective':' lambdarank', 'metric':'auc'}
bst = lgb.train(param, train_data, num_boost_round=22)
ypred = bst.predict(data.drop(['用户标识'], axis=1).values)
print roc_auc_score(data_y['是否去过迪士尼'], ypred)

In [ ]:
# 对线上结果进行输出
data_lgb = pd.read_csv('/data/topic1/result_lgb.csv')
data_lgb.rename(columns={'IMEI':'用户标识', 'SCORE':'lgb'}, inplace=True)
data_gbdt_lgb = pd.read_csv('/data/topic1/result_gbdt_lgb.csv')
data_gbdt_lgb.rename(columns={'IMEI':'用户标识', 'SCORE':'gbdt_lgb'}, inplace=True)
data_gbdt_xgb = pd.read_csv('/data/topic1/result_gbdt_xgb.csv')
data_gbdt_xgb.rename(columns={'IMEI':'用户标识', 'SCORE':'gbdt_xgb'}, inplace=True)
data = pd.merge(data_lgb, data_gbdt_lgb, on='用户标识')
data = pd.merge(data, data_gbdt_xgb, on='用户标识')
ypred = bst.predict(data.drop(['用户标识'], axis=1).values)
output = pd.DataFrame({'IMEI': data['用户标识'].values,'SCORE':ypred})
output.to_csv('/data/topic1/result_ensemble.csv', index=False, float_format='%.5f')

In [ ]:
## 其它
### 测试
temp = pd.merge(y_test, data_lgb, on='用户标识')
print roc_auc_score(temp['是否去过迪士尼'], temp['gbdt_lgb'])